## Imports

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import string
import json
import nltk
nltk.download('omw-1.4');
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sapko\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sapko\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sapko\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sapko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sapko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Crawler Component

In [3]:
# Define Seed Page
URL = "https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/"

# Define profile URL format. This was obtained by manually examining the profile pages
profile_url = "https://pureportal.coventry.ac.uk/en/persons/"

In [4]:
def get_maximum_page():
    
    first = requests.get(URL)
    soup = BeautifulSoup(first.text, 'html.parser')
    final_page = soup.select('#main-content > div > section > nav > ul > li:nth-child(12) > a')[0]['href']
    fp = final_page.split('=')[-1]
    return int(fp)
    
mx = get_maximum_page()
mx

46

web crawling for department of RCIH 

In [5]:
 #Check department
def check_department(researcher):
    
    l1 = researcher.find('div', class_='rendering_person_short')
      
    for span in l1.find_all('span'):
       
        if span.text == str('Centre for Intelligent Healthcare'):
            name = researcher.find('h3', class_='title').find('span').text
            return name
        else:
            pass

def create_csv():
     database = pd.DataFrame(columns=['Title', 'Author', 'Published', 'Link'])
     database.to_csv('database.csv')
    
def update_csv(database):
    current_data = pd.read_csv(database, index_col="Unnamed: 0")
    return current_data        

def enter_each_researchers_publication(researcher, url, df):

    new_url = url + str(researcher).replace(' ','-').lower() + '/publications/'
    page = requests.get(new_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="main-content")
    papers = results.find_all("li", class_="list-result-item")

    for paper in papers:
        title = paper.find('h3', class_='title')
        if title is not None:
            title_span = title.find('span')
            title_text = title_span.text if title_span is not None else "N/A"
        else:
            title_text = "N/A"

        author = paper.find('a', class_='link person')
        print(author)
        if author == None or author == 'None':
            continue
        else:
            author_span = author.find('span')
            author_text = author_span.text if author_span is not None else "N/A"

        date = paper.find('span', class_="date")
        date_text = date.text if date is not None else "N/A"

        link = paper.find('h3', class_='title')
        link_href = link.find('a', href=True)['href'] if link is not None else "N/A"

        # Read the existing database.csv file
        opening = pd.read_csv('data.csv', index_col="Unnamed: 0")

        # Create a new DataFrame with the data to be appended
        new_row = pd.DataFrame({'Title': [title.text],
                                'Author': [author.text],
                                'Published': [date.text],
                                'Link': [link_href]})

        # Concatenate the existing DataFrame and the new row DataFrame
        opening = pd.concat([opening, new_row], ignore_index=True)

        # Save the updated DataFrame to database.csv
        opening.to_csv('data.csv')

                

In [6]:
## Scrape function
def scrape(mx):
    df = update_csv('database.csv')
    i=0
    while True:
    
        if i > 17:
            break
            
        if i>0:
            url = URL + '?page=' + str(i)
        else:
            url = URL
    
        i = i+1
        # scraping starts here
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="main-content")
        researchers = results.find_all("li", class_="grid-result-item")

        for researcher in researchers:
            # Check if researcher has any papers
            check = researcher.find('div', class_='stacked-trend-widget')
            if check:
                name = check_department(researcher)
                if name is None:
                    pass
                else:
                    enter_each_researchers_publication(name, profile_url, df)

In [7]:
#creating csv file
create_csv()
update_csv(database='data.csv') 

%time scrape(mx) 

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
<a

In [8]:
sample_db = pd.read_csv('data.csv').rename(columns={'Unnamed: 0':'SN'})
sample_db
print(f'{sample_db.shape[0]} records were scraped')

984 records were scraped


Indexing Component

In [9]:
scraped_db = pd.read_csv('data.csv').rename(columns={'Unnamed: 0':'SN'}).reset_index(drop=True)
scraped_db.head()


,SN,Title,Author,Published,Link
0,0,Deep learning identification of coronary arter...,"Allen, J.",1 Sept 2023,https://pureportal.coventry.ac.uk/en/publicati...
1,1,Usability and preliminary efficacy of an AI-dr...,"Bul, K.",16 Apr 2023,https://pureportal.coventry.ac.uk/en/publicati...
2,2,Assessing hemodynamics from the photoplethysmo...,"Allen, J.",1 Apr 2022,https://pureportal.coventry.ac.uk/en/publicati...
3,3,Book Pitch article on Photoplethysmography: te...,"Allen, J.",25 May 2022,https://pureportal.coventry.ac.uk/en/publicati...
4,4,Ethnic Disparities in Publicly-available Pulse...,"Allen, J.",27 May 2022,https://pureportal.coventry.ac.uk/en/publicati...


In [10]:
single_row = scraped_db.loc[1,:].copy()
single_row

SN                                                           1
Title        Usability and preliminary efficacy of an AI-dr...
Author                                                 Bul, K.
Published                                          16 Apr 2023
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 1, dtype: object

Preprocess Text

In [11]:
# Remove stop words
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tp1(txt):
    # Make lowercase
    txt = txt.lower()  
    # Remove punctuation marks 
    txt = txt.translate(str.maketrans('','',string.punctuation))   
    txt = lematize(txt)
    return txt


def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax


In [12]:
# Sample title
single_row['Title']

'Usability and preliminary efficacy of an AI-driven platform supporting dietary management in diabetes: A mixed-method study'

In [13]:
# Demonstration of lowercase and punctuation removal
tp1(single_row['Title'])

'usability preliminary efficacy aidriven platform support dietary management diabetes mixedmethod study '

In [14]:
# Demonstration of lematization
lematize(tp1(single_row['Title']))


'usability preliminary efficacy aidriven platform support dietary management diabetes mixedmethod study '

#### Unprocessed

In [15]:
sample_db['Title'].iloc[5]

'Novel pulse device for diagnosis of peripheral arterial disease (NOTEPAD): ISRCTN13301188'

#### Processed

In [16]:
scraped_db['Title'].iloc[5]

'Novel pulse device for diagnosis of peripheral arterial disease (NOTEPAD): ISRCTN13301188'

### 2.1.1 Preprocess entire dataframe

In [17]:
processed_db = scraped_db.copy()

def preprocess_df(df):
    df.Title = df.Title.apply(tp1)
    df.Author = df.Author.str.lower()
    df = df.drop(columns=['Author','Published'], axis=1)
    return df
    
preprocess_df(processed_db)
processed_db.head()

,SN,Title,Author,Published,Link
0,0,deep learn identification coronary artery dise...,"allen, j.",1 Sept 2023,https://pureportal.coventry.ac.uk/en/publicati...
1,1,usability preliminary efficacy aidriven platfo...,"bul, k.",16 Apr 2023,https://pureportal.coventry.ac.uk/en/publicati...
2,2,assess hemodynamics photoplethysmogram gain in...,"allen, j.",1 Apr 2022,https://pureportal.coventry.ac.uk/en/publicati...
3,3,book pitch article photoplethysmography techno...,"allen, j.",25 May 2022,https://pureportal.coventry.ac.uk/en/publicati...
4,4,ethnic disparity publiclyavailable pulse oxime...,"allen, j.",27 May 2022,https://pureportal.coventry.ac.uk/en/publicati...


Index Construction

In [18]:
single = processed_db.loc[0,:].copy()
print(single)
indexing_trial = {}

words = single.Title.split()
SN = single.SN
word = words[0]
example = {word: [SN]}

print('=====================================================================')
print('Sample index')
print(example)

SN                                                           0
Title        deep learn identification coronary artery dise...
Author                                               allen, j.
Published                                          1 Sept 2023
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 0, dtype: object
Sample index
{'deep': [0]}


In [19]:
## Indexer Function
def apply_index(inputs, index):
    words = inputs.Title.split()
    SN = int(inputs.SN)
    for word in words:
        if word in index.keys():
            if SN not in index[word]:
                index[word].append(SN)
        else:
            index[word] = [SN]
    return index

indx = apply_index(inputs=single, index= {})

def full_index(df, index):
    for x in range(len(df)):
        inpt = df.loc[x,:]
        ind = apply_index(inputs=inpt, index=index)
    return ind

def construct_index(df, index):
    queue = preprocess_df(df)
    ind = full_index(df=queue, index=index)
    return ind

indexed = full_index(processed_db, 
                     index = {})


indexes = construct_index(df=scraped_db, 
                          index = {})

with open('indexes.json', 'w') as new_f:
    json.dump(indexes, new_f, sort_keys=True, indent=4)
    
with open('indexes.json', 'r') as file:
    data = json.load(file)

def index_2(df, x_path):
    if len(df) > 0:
        with open(x_path, 'r') as file:
            prior_index = json.load(file)
        new_index = construct_index(df = df, index = prior_index)
        with open(x_path, 'w') as new_f:
            json.dump(new_index, new_f, sort_keys=True, indent=4)

len(data)

612

In [20]:
data

{'185': [52, 175, 298, 421, 544, 667, 790, 913],
 '21st': [94, 217, 340, 463, 586, 709, 832, 955],
 '3rs': [80, 203, 326, 449, 572, 695, 818, 941],
 '3′utr': [52, 175, 298, 421, 544, 667, 790, 913],
 '7': [84, 207, 330, 453, 576, 699, 822, 945],
 'aankomende': [106, 229, 352, 475, 598, 721, 844, 967],
 'abnormality': [31, 154, 277, 400, 523, 646, 769, 892],
 'acceptance': [98, 221, 344, 467, 590, 713, 836, 959],
 'accord': [59, 182, 305, 428, 551, 674, 797, 920],
 'accuracy': [6,
  20,
  129,
  143,
  252,
  266,
  375,
  389,
  498,
  512,
  621,
  635,
  744,
  758,
  867,
  881],
 'across': [107, 230, 353, 476, 599, 722, 845, 968],
 'activation': [64, 187, 310, 433, 556, 679, 802, 925],
 'active': [49, 172, 295, 418, 541, 664, 787, 910],
 'adaptive': [61, 184, 307, 430, 553, 676, 799, 922],
 'address': [111,
  113,
  234,
  236,
  357,
  359,
  480,
  482,
  603,
  605,
  726,
  728,
  849,
  851,
  972,
  974],
 'adhd': [119,
  120,
  242,
  243,
  365,
  366,
  488,
  489,
  611,


Query Processor

In [21]:
def demonstrate_query_processing():
    sample = input('Enter Search Terms: ')
    processed_query = tp1(sample)
    print(f'Processed Search Query: {processed_query}')
    return processed_query
    
def split_query(terms):
    each = tp1(terms)
    return each.split()

dqp = demonstrate_query_processing()
dqp
print(f'Split Query: {split_query(dqp)}')

Processed Search Query: 
Split Query: []


Boolean Functionalities

In [22]:
def union(lists):
    union = list(set.union(*map(set, lists)))
    union.sort()
    return union

def intersection(lists):
    intersect = list(set.intersection(*map(set, lists)))
    intersect.sort()
    return intersect


Search Engine Function

In [23]:
#Search engine function
def vertical_search_engine(df, query, index=indexes):
    query_split = split_query(query)
    retrieved = []
    for word in query_split:
        if word in index.keys():
            retrieved.append(index[word])              
    # Ranked Retrieval
    if len(retrieved)>0:
        high_rank_result = intersection(retrieved)
        low_rank_result = union(retrieved) 
        c = [x for x in low_rank_result if x not in high_rank_result]      
        high_rank_result.extend(c)
        result = high_rank_result
        
        final_output = df[df.SN.isin(result)].reset_index(drop=True)
    
        # Return result in order of Intersection ----> Union
        dummy = pd.Series(result, name = 'SN').to_frame()
        result = pd.merge(dummy, final_output, on='SN', how = 'left')   
    else:
        result = 'No result found'
    
    return result

def test_search_engine():
    xtest = scraped_db.copy()
    query = input("Enter your search query: ")
    return vertical_search_engine(xtest, query, indexed)
    
test_search_engine()

'No result found'

In [24]:
def final_engine(results):
    if type(results) != 'list':
        return results
        #print(results)
    else:
        for i in range(len(results)):
            printout = results.loc[i, :]
            

In [25]:
scraped_db['Author'].iloc[24]

'zheng, d.'

In [26]:
final_engine(test_search_engine())

'No result found'

Schedule Crawler for every week

To demonstrate a weekly scheduled crawling, the following parameters are defined:


In [27]:
days = 0
interval = 7
while days <= 1:
    scrape(mx)
    print(f"Crawled at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f'Next crawl scheduled after {interval} days')
    time.sleep(interval)
    days = days + 1

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
<a

Search Engine User Interface

In [28]:
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk 
from tkinter import scrolledtext
from pandastable import Table, TableModel
from contextlib import suppress
import warnings
warnings.filterwarnings('ignore')

image1 = Image.open('logo.png')
resized_image1 = image1.resize((500,200), Image.ANTIALIAS)

def new_gui(image1):
    window = Tk()
    window.configure(bg='#98AFC7')
    window.title("IR Search Engine")
    window.geometry('1000x600')
    
    lbl1 = Label(window, text="Enter your query here", bg="#87CEFA",font=("Cambria", 12), padx=5, pady=5)
    lbl1.grid(column=0, row=1)
    
    img = ImageTk.PhotoImage(image1)
    
    lbl2 = Label(image=img)
    lbl2.image = img
    lbl2.grid(column=1, row=3, padx=5, pady=5)
    
    query = Entry(window, bg="#87CEFA", font=("Cambria bold", 12), width=60)
    query.grid(column=1, row=1,  padx=5, pady=5)
    
    results = Canvas(window, height=30, width=600)
    results.grid(column=1, row=2, padx=5, pady=5)
    
    # Entry
    def getInputBoxValue():
        userInput = query.get()
        return userInput

    # Button
    def clicked():
        search()
           
    def no_result():
        messagebox.showwarning("Warning", "No results found. Please try again")
        
    def search():
        xtest = scraped_db.copy()
        q = query.get()
        f = Frame(window)
        df = vertical_search_engine(xtest, q, indexed)
        if type(df) == str:
            no_result()
        else:
            pt = Table(results)
            try:
                table = pt = Table(results, dataframe=df)
                pt.show()
            except AttributeError:
                pass
            
    def close_window():
        if messagebox.askokcancel("Quit", "Quit Programme?"):
            window.destroy()
        
    btn = Button(window, text="Search",font=("Cambria", 12), bg="#87CEFA", command=clicked)
    btn.grid(column=2, row=1)
    
    window.protocol("WM_DELETE_WINDOW", close_window)       
    window.mainloop()

In [30]:
new_gui(resized_image1)